트랜스포머 아키텍처 = 언어를 이해하는 인코더 + 언어를 생성하는 디코더
RNN -> attention is all new need -> 높은 성능과 빠른 학습속도
순차적으로 하나씩 입력을 받는 RNN은 학습 속도가 느리고, 입력이 길어지면 먼저 입력한 토큰의 정보가 희석되면서 성능이 떨어진다. 또한 성능을 높이기 위해 층을 깊이 쌓으면 그레디언트 소실이나 증폭(gradient vanishing, exploding)이 발생하며 학습이 불안전해진다.
셀프 어텐션: 입력된 문장 내의 각 단어가 서로 어떤 관련이 있는지 계산해서 각 단어의 표현을 조정하는 역할
    확장성: 더 깊은 모들을 만들어도 학습이 잘 되고 동일한 블록을 반복해 사용하기 때문에 확장이 용이하다.
    효율성: 학습할 때 병렬 연산이 가능하기 때문에 학습 시간이 단축된다.
    더 긴 입력처리: 입력이 길어져도 성능이 거의 떨어지지 않는다.
입력을 임베딩 층을 통해 숫자 집합인 임베딩으로 변환하고 위치 인코딩(positional encoding) 층에서 문장의 위치 정보를 더한다. 인코더에서는 층 정규화(layer normalization), 멀티 헤드 어텐ㅅ녀(multi head attention), 피드 포워드(feed forward)층을 거치며 문장을 이해하고 디코더로 전달한다. 디코더에서는 인코더에서와 유사하게 층 정규화, 멀티 헤드 어텐션 연산을 수행하면서 크로스 어텐션 연산을 통해 인코더가 전달한 데이터를 출력화 함께 종합해서 피드 포워드 층을 거쳐 결과를 생성한다.


텍스트를 모델에 입력할 수 있는 숫자형 데이터인 임베딩으로 변화하기 위해서는 세가지 과정을 거쳐야 한다. 
    1. 텍스트를 적절한 단위로 달라 숫자형 아이디를 부여하는 토큰화
    2. 토큰 아이디를 토큰 임베딩 층을 통해 여러 숫자 집한인 토큰 임베딩으로 변환
    3. 위치 인코딩 층을 통해 토큰의 위치 정보를 담고 있는 위치 임베딩을 추가해 최정적으로 모델에 입력할 임베딩을 만든다.


토큰화: 텍스트를 적절한 단위로 나누고 숫자 아이디를 부여하는 것.

큰 단위를 기준으로 토큰화 할수록 텍스트의 의미가 잘 유지된다는 장점이 있지만, 사전의 크기가 커진다는 단점이 있다.
이전에 본 적 없는 새로운 단어는 사전에 없기 때문에 처리하지 못하는 OOV(Out Of Vocabulary) 문제가 자주 발생한다.
작은 단위로 토큰화 하는 경우 사전의 크기가 작고 OOV 문제를 줄일 수 있지만 텍스트의 의미가 유지되지 않는다는 단점이 있다.
두 방법 모두 장단점이 뚜렷하기 때문에 최근에는 데이터에 등장하는 빈도에 따라 토큰화 단위를 결정하는 서브워드(subword) 토큰화 방식을 사용한다.
서브워드 토큰화 방식에서는 자주 나오는 단어는 단어 단위 그대로 유지하고 가끔 나오는 단어는 더 작은 단위로 나눠 텍스트의 의미를 최대한 유지하면서 사전의 크기는 작고 효율적으로 유지할 수 있다.
     ex) '안녕', '대한민국' 같이 자주 나오는 단어는 그대로 단어 형태를 유지, 특수문자, 이모지 등은 작게 나눠 사전 크기가 커지지 않도록 한다.

sadfafa

In [1]:
import torch
import torch.nn as nn

In [2]:
input_text = "나는 최근 파리 여행을 다녀왔다."
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다.']


In [3]:
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다.': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다.'}


In [4]:
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_ids:  [0, 1, 2, 3, 4]


In [5]:
print(input_text_list)
print(str2idx)
print(idx2str)
print(input_ids)

['나는', '최근', '파리', '여행을', '다녀왔다.']
{'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다.': 4}
{0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다.'}
[0, 1, 2, 3, 4]


모델이 텍스트 데이터를 처리하기 위해서는 입력으로 들어오는 토큰과 토큰 사이의 관계를 계산할 수 있어야 한다. 토큰과 토큰 사이의 관계를 계산하기 위해서는 토큰의 의미를 숫자로 나타낼 수 있어야 하는데 토큰 아이디는 하나의 숫자일 뿐이므로 토큰의 의미를 담을 수 없다. 의미를 담기 위해서는 최소 2개 이상의 숫자 집합인 벡터여야 한다. nn.Embedding 클래스를 사용하면 토큰 아이디를 토큰 임베딩으로 변환할 수 있다. 이 코드에서는 nn.Embedding 클래스에 사전 크기가 len(str2idx)이고 embedding_dim차원의 임베딩을 생성하는 임베딩 층인 embed_layer를 만들고 입력 토큰을 임베딩 층을 통해 임베딩으로 변환한다. embedding_dim을 16으로 설정해 토큰 하나를 16차원의 벡터로 변환한다. 출력 결과를 보면 1개의 문장이고, 5개의 토큰이 있고, 16차원의 임베딩이 생성됐음을 확인할 수 있다.

In [6]:
embedding_dim = 16
embed_layer = torch.nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
print(input_embeddings.shape)

torch.Size([1, 5, 16])


위의 임베딩 층은 토큰의 의미를 담아 벡터로 변환하지 않는다. 지금의 임베딩 층은 그저 입력 토큰 아이디를 16차원의 임의의 숫자 집합으로 바꿔줄 뿐이다. 임베딩 층이 단어의 의미를 담기 위해서는 딥러닝 모델이 학습 모델로 훈련되어야 한다.
모델이 특정 작업을 잘 수행하도록 학습하는 과정에서 데이터의 의미를 잘 담은 임베딩을 만드는 방법도 함께 학습한다.

absolute position encoding vs relative position encoding
RNN과 트랜스포머의 가장 큰 차이점은 입력을 순차적으로 처리하는지 여부다. RNN은 입력을 순차적으로 처리하는데, 그렇기 때문에 자연스럽게 입력 데이터의 순서 정보가 고려된다. 트랜스포머는 순차적인 처리 방식을 버리고 모든 입력을 동시에 처리하는데, 그 과정에서 순서 정보가 사라지게 된다. 하지만 텍스트에서 순서는 매우 중요한 정보이기 때문에 추가해 줘야 하는데, 그 역할을 위치 인코딩이 담당한다.
Attention is ALl you need 논문에서는 사인과 코사인을 활용한 수식을 통해 위치에 대한 정보를 입력했다. 하지만 그 이후에는 위치 인코딩도 위치에 따른 임베딩 층을 추가해 학습 데이터를 통해 학습하는 방식을 많이 활용하고 있다. 수식을 통해 위치 정보를 추가하는 방식이나 임베딩으로 위치 정보를 학습하는 방식 모두 결국 모델로 추론을 수행하는 시점에서는 입력 토큰의 위치에 따라 고정된 임베딩을 더해주기 때문에, 이를 절대적 위치 인코딩이라고 부른다.

절대적 위치 인코딩 방식은 간단하게 구현할 수 있다는 장점이 있지만 토큰과 토큰 사이의 상대적인 위치 정보는 활용하지 못하고, 학습 데이터에서 보기 어려웠떤 긴 텍스트를 추론하는 경우에는 성능이 떨어진다는 문제가 있어 최근에는 상대적 위치 인코딩 방식도 많이 활용한다.

절대적 위치 인코딩 중 위치 정보를 학습하는 방식을 코드로는 아래와 같이 새로운 임베딩 층을 하나 추가하고 위치 인덱스(psotion_ids)에 따라 임베딩을 더하도록 구현할 수 있다. 최대 토큰수(max_position)을 12로 설정하여 위치 인코딩을 생성하는 위치 임베딩 층(position_embed_lay)을 정의한다. 위치 아이디(position_ids)에는 0부터 입력 토큰의 수까지 1씩 증가하도록 데이터를 생성한다. position_ids를 위치 임베딩 층에 입력해 위치 인코딩(position_encodings)을 생성하고 토큰 임베딩(token_embedings)에 위치 인코딩을 더해 모델에 입력할 최종 입력 임베딩(input_embeddings)을 준비한다.

In [7]:
embedding_dim = 16
max_position = 12
embed_layer = torch.nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = torch.nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings + position_encodings

print(position_ids)
print(position_encodings.shape)
print(token_embeddings.shape)

tensor([[0, 1, 2, 3, 4]])
torch.Size([1, 5, 16])
torch.Size([1, 5, 16])


"나는 최근 파리 여행을 다녀왔따"라는 문장을 단어 단위로 토큰화를 수행하고 각 토큰에 토큰 아이디와 위치 아이디를 부여했다. 
실제 토큰 아이디와 위치 아이디 모두 [0,1,2,3,4]의 값을 갖는데, 값은 동일하지만 토큰 아이디는 사전에 저장된 토큰의 고유한 아이디를 의믜하고 위치 아이디는 토큰의 위치를 의미한다. 예시 데이터이기에 같게 표현된것이고 일반적으로는 같지 않다. 하나의 숫자로된 숫자 아이디는 데이터의 의미를 담을 수 없기 때문에 의미를 담을 수 있또록 토큰 아이디와 위치 아이디를 토큰 임베딩층과 위치 인코딩 층을 통해 토큰 임베딩과 위치 임베딩으로 변환한다.

트랜스포머 아키텍처의 핵심은 논무의 제목에서도 알 수 있듯이 '어텐션'이다. 어텐션의 사전적 의미는 '주의'라고 번역할 수 있는데, 텍스트를 처리하는 관점에서는 입력한 텍스트에서 어떤 단어가 서로 관련되는지 '주의를 기울여' 파악한다는 의미로 이해할 수 있다.

트랜스포머 아키텍처를 개발한 연구진은 정보 분야에서 쿼리, 키, 값이라는 개념을 도입했다. 쿼리, 키, 값에 대한 각각의 가중치를 weight_q, weight_k, weight_v로 생성하고 입력으로 준비한 input_embedding을 선형 층에 통과시켜 쿼리, 키, 값을 생성한다.

단계별로 먼저 쿼리와 키를 곱한다. 이때 분산이 커지는 것을 방지하기 위해 임베딩 차원 수의 제곱근으로 나눈다. 다음으로 쿼리와 키를 곱해 계산한 스코어를 합이 1이 되도록 소프트맥스를 취해 가중치로 바꾼다. 마지막으로 가중치와 값을 곱해 입력과 동일한 형태를 반환한다.

In [8]:
head_dim = 16

weight_q = torch.nn.Linear(embedding_dim, head_dim)
weight_k = torch.nn.Linear(embedding_dim, head_dim)
weight_v = torch.nn.Linear(embedding_dim, head_dim)

querys = weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)


In [9]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return weights @ values

In [10]:
print(input_embeddings.shape)
after_attention_embeddings = compute_attention(querys, keys, values)

print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])
torch.Size([1, 5, 16])


In [11]:

class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        self.weight_v = nn.Linear(token_embed_dim, head_dim)
        
    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values),
            is_causal = self.is_causal
        )
        return outputs
        

In [12]:
attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(querys, keys, values)

트랜스포머 아키텍처를 고안한 논문 저자들은 한번에 하나의 어텐션 연산만 수행하는게 아니라 여러 어텐션 연산을 동시에 적용하면 성능을 더 높일 수 있다는 사실을 발견했다. 이를 멀티 헤드 어텐션이라고 한다. 직관적으로 이해하자면, 토큰 사이의 관계를 한가지 측면에서 이해하는 것보다 여러 측면을 동시에 고려할 때 언어나 문장에 대한 이해도가 높아질 것이다.

AttentionHead와 대부분의 코드가 동일한데 헫드의 수만큼 연산을 수행하기 위해 쿼리, 키, 값을 n_head개로 쪼개고 각각의 어텐션을 계산한 다음 입력과 같은 형태로 다시 변환한다. 마지막으로 선형층을 통과시키고 최종 결과를 반환한다.

In [13]:
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)
        
    def forward(self, querys, keys, values):
        B, T, C = querys.size()
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        
        attention = compute_attention(querys, keys, values, self.is_causal)
        output = attention.transpose(1, 2).contiguous().view(B, T, C)
        output = self.concat_linear(output)
        return output

In [14]:
n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

데이터를 정규화하여 모든 입력 변수가 비슷한 범위의 분포를 갖도록 조정하면 모델은 각 입력 변수의 중요성을 적절히 반영하여 좀 더 정확한 예측을 할 수 있게 된다.

자연어 처리에서는 입력으로 들어가는 문장의 길이가 다양한데, 배치 정규화를 사용할 경우 정규화에 포함되는 데이터의 수가 제각각이라 정규화 효과를 보장하기 어렵기 때문에 배치 정규화를 사용하지 않는다.

층 정규화는 이런 단점을 보완할 수 있도록 각 토큰 임베딩의 평균과 표준편차를 구해 정규화를 수행한다. 문장별로 실제 데이터의 수가 다르더라도 각각의 토큰 임베딩별로 정규화를 수행하기 때문에 정규화 효과에 차이가 없다.

On Layer Normalizzation in the Transformer Architecture 논문에서 먼저 층 정규화를 적용하고 어텐션과 피드 포워드 층을 통화했을 때 학습이 더 안정적이라는 사실이 확인되어 post norm 대신 pre norm이 주로 활용된다.

In [15]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape

torch.Size([1, 5, 16])

In [16]:
norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

(tensor([[-1.4901e-08, -2.6077e-08, -2.6077e-08, -1.1176e-08,  1.4901e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

피드 포워드 층은 데이터의 특징을 학습하는 완전 연결 층을 말한다. 멀티 헤드 어텐션이 단어 사의의 관계를 파악하는 역할이라면 피드 포워드 층은 입력 텍스트 전체를 이해하는 역할을 담당한다.

In [17]:
 class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = nn.GELU()
        self.norm = nn.LayerNorm(d_model)
        
    def forard(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
        x = self.dropout2(x)
        return x

인코더 구조에 따라 입력인 src를 self.norm을 통해 층 정규화를 취하고 멀티 헤드 어텐션 클래스를 인스턴스화한 self.attn을 통해 멀티 헤드 어텐션 연산을 수행한 후 잔차 연결을 위해 어텐션 결과에 드롭아웃을 위한 self.dropout1과 입력을 더해준다. 마지막으로 self.feed_forward를 통해 피드 포원드 연산을 취한다.

In [18]:
class TransformerEncoderLayer(nn.Module):
    def __init(self, d_model, dim_feedforward, dropout):
        super().__init__(self)
        self.attn = MultiheadAttention(d_model, d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)
        
    def forard(self, src):
        norm_x = self.norm1(src)
        attn_output = self.attn(norm_x, norm_x, norm_x)
        x = src + self.dropout1(attn_output)
        
        x = self.feed_forward(x)
        return x

인코더 층을 N_e번 반복되도록 코드로 구현한다. TransformerEncoder 클래스에서는 인자로 전달받은 encoder_layer를 get_clones 함수를 통해 num_layers번 반복하여 nn.ModuleList에 넣고 forward 메서드에서 for 문을 통해 순회하면서 인코더 층 연산을 반복 수행하도록 만든다.

In [19]:
import copy
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
        
    def forward(self, src):
        output = src
        for mod in self.layers:
            for mod in self.layers:
                output = mod(output)
            return output

디코더는 인코더와 비교할 때 두가지 부분에서 차이가 있다. 먼저, 인코더는 멀티 헤드 어텐션을 사용하지만 디코더 블록에서는 마스크 멀티 헤드 어텐션을 사용한다. 디코더는 생성을 담당하는 부분으로, 사람이 글을 쓸 때 앞 단어부터 순차적으로 작성하는 것처럼 트랜스포머 모델도 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성한다. 이렇게 순차적으로 생성해야 하는 특징을 인과적(causal) 또는 자기회귀적(auto-regressive)이라고 한다.

실제 텍스트를 생성할 때 디코더는 이전까지 생성한 텍스트만 확인할 수 있다. 그런데 학습할 때는 인코더와 디코더 모두 완성된 텍스트를 입력으로 받는다. 따라서 어텐션을 그대로 할용할 경우 미래 시점에 작성해야 하는 텍스트를미리 확인하게 되는 문제가 생긴다. 이를 막기 위해서는 특정 시점에서는 그 이전에 생성된 토큰까지만 확인할 수 있도록 마스크를 추가한다. 

is_causal이 참일 때는 torch.ones로 모두 1인 행렬에 tril 함수를 취해 가운데 행렬과 같이 대각선 아래 부분만 1로 유지되고 나머지는 음의 무한대로 변경해 마스크를 생성한다. 마스크를 어텐션 스코어 행렬에 곱하면 행렬의 대각선 아랫부분만 어텐션 스코어가 남고 위쪽은 음의 무한대가 된다. 소프트 맥스를 취하면 음의 무한대인 대각선 윗부분은 가중치가 0이 된다.

In [20]:
def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    if is_causal:
        query_length = querys.size(2)
        key_length = keys.size(2)
        temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
        scores = scores.masked_fill(temp_mask == False, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights @ values
        

인코더의 결과를 디코더가 활용하는 크로스 어텐션 연산을 해야 한다. 이때 쿼리는 디코더의 잠재 상태를 사용하고 키와 값은 인코더의 결과를 사용한다

인코더의 결과를 forward 메서드에 encoder_output이라는 이름의 인자로 넣을 수 있도록 했는데 self.multihead_attn을 통해 크로스어텐션 연산을 수행한다.

디코더는 인코더와 마찬가지로 디코더 층을 여러 번 쌓아 만든다. 디코더 층을 N번 반복하여 nn.ModuleList를 활용해 순회하도록 한다.

In [21]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, d_model, n_head)
        self.multihead_attn = MultiheadAttention(d_model, d_model, n_head)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, tgt, encoder_output, is_causal=True):
        x = self.norm1(tgt)
        x = x + self.dropout1(self.self_attn(x,x,x,is_causal=is_causal))
        x = self.norm2(x)
        x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
        x = self.feed_forward(x)
        return x

In [22]:
class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
    
    def forward(self, tgt, src):
        output = tgt
        for mod in self.layers:
            output = mod(output)
        return output